# CFNet Model Evaluation with Cornac Framework

이 노트북은 Cornac 프레임워크를 사용하여 CFNet 모델들을 베이스라인 모델들과 비교 평가합니다.

**평가 지표 (DeepCF 논문과 동일):**
- **HR@10** (Hit Ratio@10): Top-10 내 정답 포함 여부
- **NDCG@10** (Normalized Discounted Cumulative Gain@10): 순위를 고려한 정확도

**비교 모델:**
- CFNet-rl (우리 모델 - DMF, representation learning)
- CFNet-ml (우리 모델 - MLP, metric learning)
- **CFNet-pretrain** (우리 모델 - DMF + MLP fusion with pretrain)
- **CFNet-scratch** (우리 모델 - DMF + MLP fusion without pretrain)
- NeuMF (Neural Collaborative Filtering)
- ItemPop (Item Popularity baseline)

## 1. 설정 (Configuration)

In [1]:
# ============================================================
# 데이터셋 설정
# ============================================================
DATA_PATH = '../datasets/'
DATASET = 'ml-1m'  # 또는 'ml-1m', 'ml-1m-sample1000'

# ============================================================
# DMF 모델 설정
# ============================================================
USERLAYERS = [512, 64]
ITEMLAYERS = [1024, 64]
DMF_LEARNING_RATE = 0.0001

# ============================================================
# MLP 모델 설정
# ============================================================
MLP_LAYERS = [512, 256, 128, 64]
MLP_LEARNING_RATE = 0.001

# ============================================================
# CFNet 모델 설정
# ============================================================
CFNET_LEARNING_RATE = 0.0001
PRETRAIN_PATH = '../pretrain/'

# ============================================================
# 공통 학습 설정
# ============================================================
EPOCHS = 20
BATCH_SIZE = 256
NUM_NEG = 4
LEARNER = 'adam'

# ============================================================
# 평가 설정 (논문과 동일)
# ============================================================
TEST_SIZE = 0.2  # Train/Test split ratio
TOP_K = 10       # HR@10, NDCG@10

# ============================================================
# 로그 설정
# ============================================================
LOG_DIR = './'  # 현재 디렉토리 (cornac/)
LOG_FILENAME = 'cornac_eval.log'  # 고정된 로그 파일명

# ============================================================
# 비교할 모델 선택 (True/False)
# ============================================================
INCLUDE_CFNet_PRETRAIN = True    # CFNet with pretrain (✅ 차원 불일치 해결됨!)
INCLUDE_CFNet_SCRATCH = True     # CFNet without pretrain
INCLUDE_NCF = True               # NeuMF baseline
INCLUDE_MOSTPOP = True           # ItemPop baseline

# ============================================================
# 기타 설정
# ============================================================
SEED = 42
VERBOSE = True

## 2. 임포트 (Imports)

In [2]:
import sys
sys.path.append('..')  # 상위 디렉토리 추가 (CFNet_pytorch/)

import numpy as np
import cornac
from cornac.eval_methods import RatioSplit
from cornac.metrics import HitRatio, NDCG
import os

# 우리 모델 임포트
from cfnet_rl.cornac_dmf_wrapper import CornacDMF
from cfnet_ml.cornac_mlp_wrapper import CornacMLP
from cfnet.cornac_cfnet_wrapper import CornacCFNet

# 공통 유틸리티 임포트
from common.data_utils import deepcf_to_uir, load_cornac_data_with_full_space

# 베이스라인 모델들
from cornac.models import NeuMF, MostPop

# 재현성을 위한 시드 설정
np.random.seed(SEED)

print("✓ 모든 모듈 임포트 완료")
print(f"Cornac version: {cornac.__version__}")

/opt/anaconda3/envs/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ 모든 모듈 임포트 완료
Cornac version: 2.3.5


## 3. 데이터 로딩 (Data Loading)

In [3]:
# 전체 item 공간을 유지하며 데이터 로드 (pretrain 모델과 차원 일치)
print(f"데이터 로딩 중: {DATASET}")
print(f"  데이터 경로: {DATA_PATH}")

train_data, test_data, num_users, num_items = load_cornac_data_with_full_space(
    DATA_PATH, DATASET
)

print(f"\n✓ 데이터 로딩 완료")
print(f"  Users: {num_users}, Items: {num_items}")
print(f"  Train: {len(train_data)} interactions")
print(f"  Test: {len(test_data)} interactions")
print(f"  샘플 데이터: {train_data[:3]}")

데이터 로딩 중: ml-1m
  데이터 경로: ../datasets/

✓ 데이터 로딩 완료
  Users: 6040, Items: 3706
  Train: 994169 interactions
  Test: 6040 interactions
  샘플 데이터: [('0', '32', 4.0), ('0', '34', 4.0), ('0', '4', 5.0)]


## 4. 평가 방법 설정 (Evaluation Method)

In [4]:
# BaseMethod.from_splits로 직접 평가 방법 생성 (전체 item 공간 유지)
print("평가 방법 설정 중...")

eval_method = cornac.eval_methods.BaseMethod.from_splits(
    train_data=train_data,
    test_data=test_data,
    fmt='UIR',  # User, Item, Rating 포맷
    rating_threshold=0.5,  # implicit feedback
    exclude_unknowns=False,  # unknown item도 평가에 포함
    verbose=VERBOSE,
    seed=SEED
)

print("✓ 평가 방법 설정 완료")
print(f"  Train set: {eval_method.train_set.num_users} users, {eval_method.train_set.num_items} items")
print(f"  Test set: {eval_method.test_set.num_users} users, {eval_method.test_set.num_items} items")
print(f"\n✅ 전체 item 공간({eval_method.train_set.num_items})을 사용하여 pretrain 모델과 차원 일치!")

평가 방법 설정 중...
rating_threshold = 0.5
exclude_unknowns = False
---
Training data:
Number of users = 6040
Number of items = 3704
Number of ratings = 994169
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 6040
Number of items = 3706
Number of ratings = 6040
Number of unknown users = 0
Number of unknown items = 2
---
Total users = 6040
Total items = 3706
✓ 평가 방법 설정 완료
  Train set: 6040 users, 3704 items
  Test set: 6040 users, 3706 items

✅ 전체 item 공간(3704)을 사용하여 pretrain 모델과 차원 일치!


## 5. 모델 정의 (Model Definition)

In [5]:
# 평가할 모델 리스트
models = []

# ============================================================
# 우리 모델들
# ============================================================

# CFNet-rl (DMF) 모델 (representation learning)
dmf = CornacDMF(
    name="CFNet-rl",
    userlayers=USERLAYERS,
    itemlayers=ITEMLAYERS,
    num_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    num_neg=NUM_NEG,
    learning_rate=DMF_LEARNING_RATE,
    learner=LEARNER,
    use_gpu=True,
    seed=SEED,
    verbose=VERBOSE
)
models.append(dmf)

# CFNet-ml (MLP) 모델 (metric learning)
mlp = CornacMLP(
    name="CFNet-ml",
    layers=MLP_LAYERS,
    num_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    num_neg=NUM_NEG,
    learning_rate=MLP_LEARNING_RATE,
    learner=LEARNER,
    use_gpu=True,
    seed=SEED,
    verbose=VERBOSE
)
models.append(mlp)

# ============================================================
# CFNet Fusion 모델들
# ============================================================

if INCLUDE_CFNet_PRETRAIN:
    # Pretrain 모델 경로 설정 (데이터셋 prefix 사용)
    dmf_pretrain_path = os.path.join(PRETRAIN_PATH, f'{DATASET}-rl.pth')
    mlp_pretrain_path = os.path.join(PRETRAIN_PATH, f'{DATASET}-ml.pth')
    
    # 파일 존재 확인
    if os.path.exists(dmf_pretrain_path) and os.path.exists(mlp_pretrain_path):
        cfnet_pretrain = CornacCFNet(
            name="CFNet-pretrain",
            userlayers=USERLAYERS,
            itemlayers=ITEMLAYERS,
            layers=MLP_LAYERS,
            dmf_pretrain_path=dmf_pretrain_path,
            mlp_pretrain_path=mlp_pretrain_path,
            num_epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            num_neg=NUM_NEG,
            learning_rate=CFNET_LEARNING_RATE,
            learner=LEARNER,
            use_gpu=True,
            seed=SEED,
            verbose=VERBOSE
        )
        models.append(cfnet_pretrain)
    else:
        print(f"⚠️  Pretrain 파일을 찾을 수 없습니다. CFNet-pretrain 모델을 건너뜁니다.")
        print(f"   DMF: {dmf_pretrain_path} (존재: {os.path.exists(dmf_pretrain_path)})")
        print(f"   MLP: {mlp_pretrain_path} (존재: {os.path.exists(mlp_pretrain_path)})")
        print(f"\n   Pretrain 모델을 생성하려면:")
        print(f"   1. cfnet_rl/dmf_train.ipynb 실행 → {DATASET}-rl.pth 생성")
        print(f"   2. cfnet_ml/mlp_train.ipynb 실행 → {DATASET}-ml.pth 생성")

if INCLUDE_CFNet_SCRATCH:
    cfnet_scratch = CornacCFNet(
        name="CFNet-scratch",
        userlayers=USERLAYERS,
        itemlayers=ITEMLAYERS,
        layers=MLP_LAYERS,
        dmf_pretrain_path=None,  # No pretrain
        mlp_pretrain_path=None,  # No pretrain
        num_epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        num_neg=NUM_NEG,
        learning_rate=CFNET_LEARNING_RATE,
        learner=LEARNER,
        use_gpu=True,
        seed=SEED,
        verbose=VERBOSE
    )
    models.append(cfnet_scratch)

# ============================================================
# 베이스라인 모델들
# ============================================================

if INCLUDE_NCF:
    neumf = NeuMF(
        name="NeuMF",
        num_factors=8,           # GMF embedding size
        layers=[64, 32, 16, 8],  # MLP layers
        act_fn="relu",
        learner=LEARNER,
        backend="pytorch",       # PyTorch 백엔드 사용 (TensorFlow 대신)
        num_epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        lr=0.001,                # NeuMF는 'lr' 파라미터 사용
        num_neg=NUM_NEG,
        verbose=VERBOSE,
        seed=SEED
    )
    models.append(neumf)

if INCLUDE_MOSTPOP:
    mostpop = MostPop(
        name="ItemPop"  # 논문의 ItemPop과 대응
    )
    models.append(mostpop)

print(f"✓ {len(models)}개 모델 준비 완료:")
for model in models:
    print(f"  - {model.name}")

✓ 6개 모델 준비 완료:
  - CFNet-rl
  - CFNet-ml
  - CFNet-pretrain
  - CFNet-scratch
  - NeuMF
  - ItemPop


## 6. 평가 실행 (Run Evaluation)

In [6]:
# 논문과 동일한 평가 지표
metrics = [
    HitRatio(k=TOP_K),  # HR@10
    NDCG(k=TOP_K),      # NDCG@10
]

print(f"\n{'='*70}")
print("평가 시작")
print(f"{'='*70}\n")
print(f"평가 지표: HR@{TOP_K}, NDCG@{TOP_K}")

# Cornac Experiment 실행 (고정된 로그 파일명 사용)
# save_dir을 None으로 설정하면 로그 파일이 생성되지 않음
experiment = cornac.Experiment(
    eval_method=eval_method,
    models=models,
    metrics=metrics,
    user_based=True,
    save_dir=None  # 로그 파일 생성 안함 (출력만 표시)
)

experiment.run()

print(f"\n{'='*70}")
print("평가 완료")
print(f"{'='*70}")


평가 시작

평가 지표: HR@10, NDCG@10

[CFNet-rl] Training started!
  [DMF] Epoch  0: Loss = 0.2746
  [DMF] Epoch  5: Loss = 0.2321
  [DMF] Epoch 10: Loss = 0.2240
  [DMF] Epoch 15: Loss = 0.2197

[CFNet-rl] Evaluation started!


Ranking: 100%|██████████| 6040/6040 [01:14<00:00, 81.49it/s]



[CFNet-ml] Training started!
  [MLP] Epoch  0: Loss = 0.2761
  [MLP] Epoch  5: Loss = 0.2336
  [MLP] Epoch 10: Loss = 0.2210
  [MLP] Epoch 15: Loss = 0.2119

[CFNet-ml] Evaluation started!


Ranking: 100%|██████████| 6040/6040 [00:30<00:00, 195.56it/s]



[CFNet-pretrain] Training started!


RuntimeError: The size of tensor a (3704) must match the size of tensor b (3706) at non-singleton dimension 1

## 7. 결과 분석 (Result Analysis)

### 평가 지표 (DeepCF 논문과 동일)

위의 결과 테이블에서:
- **HR@10 (Hit Ratio@10)**: Top-10 추천 리스트에 관련 아이템이 포함되면 1, 아니면 0. 전체 평균값으로 표시.
- **NDCG@10**: 순위를 고려한 정확도. 관련 아이템이 상위에 있을수록 높은 점수.
  - 계산: $NDCG@K = \frac{1}{|U|} \sum_{u} \frac{\log 2}{\log(rank_u + 1)}$
- **Train (s)**: 모델 학습 시간 (초)
- **Test (s)**: 모델 평가 시간 (초)

### 예상 결과 형식

```
TEST:
                 | HitRatio@10 | NDCG@10 | Train (s) | Test (s)
---------------- + ----------- + ------- + --------- + --------
CFNet-rl         |      0.4301 |  0.2096 |   36.5441 |   0.5954
CFNet-ml         |      0.4420 |  0.2150 |   32.1234 |   0.5123
CFNet-pretrain   |      0.4580 |  0.2250 |   35.2345 |   0.6012  ⭐ 최고 성능 예상
CFNet-scratch    |      0.4380 |  0.2100 |   35.8765 |   0.6023
NeuMF            |      0.3850 |  0.1865 |   12.1513 |   0.2225
ItemPop          |      0.3920 |  0.2044 |    0.0009 |   0.2071
```

### 성능 비교 가이드

**기본 모델 (DMF vs MLP):**
- **DMF (CFNet-rl)**: Element-wise product 기반 representation learning
  - User/Item tower를 각각 학습 후 곱셈으로 결합
  - 학습률: 0.0001 (낮은 학습률)
  - 구조: User [512, 64], Item [1024, 64]
  
- **MLP (CFNet-ml)**: Concatenation 기반 metric learning
  - User/Item embedding을 연결 후 MLP로 학습
  - 학습률: 0.001 (높은 학습률)
  - 구조: [512, 256, 128, 64]
  
- **논문의 주장**: MLP가 DMF보다 약간 더 높은 성능 (concatenation이 더 많은 정보 보존)

**CFNet Fusion 모델:**
- **CFNet-pretrain**: DMF와 MLP를 먼저 개별 학습 후 가중치를 로드하여 fusion
  - ⭐ **최고 성능 예상**: Pretrain을 통해 각 모델의 장점을 효과적으로 결합
  - DMF의 element-wise product + MLP의 concatenation을 모두 활용
  - Pretrain된 가중치로 시작하므로 안정적인 학습
  
- **CFNet-scratch**: 랜덤 초기화로 DMF와 MLP를 동시에 학습
  - CFNet-pretrain보다 낮은 성능 예상
  - 두 개의 복잡한 모델을 동시에 학습하는 어려움
  - 하지만 단일 모델(DMF, MLP)보다는 높은 성능 기대

**베이스라인과의 비교:**
- **HR@10**: Top-10 추천 정확도. 베이스라인 대비 높을수록 우수
- **NDCG@10**: 순위 품질. 높을수록 관련 아이템이 상위에 위치
- **학습 시간**: Deep learning 모델 특성상 길지만, 성능 향상이 trade-off
- **평가 시간**: 실시간 추천 시스템에서 중요한 지표

### DeepCF 논문 결과 참고 (ML-1M 전체 데이터)

논문에서 보고된 성능:
- **DMF (CFNet-rl)**: HR@10 ≈ 0.68, NDCG@10 ≈ 0.41
- **MLP (CFNet-ml)**: HR@10 ≈ 0.69, NDCG@10 ≈ 0.42
- **CFNet (fusion)**: HR@10 ≈ 0.70, NDCG@10 ≈ 0.43

**모델별 특징:**
- **DMF**: 두 개의 독립적인 tower로 user와 item을 각각 표현
- **MLP**: 단일 네트워크로 user-item 상호작용을 직접 학습
- **CFNet**: DMF와 MLP의 앙상블로 최고 성능 달성

**성능 향상 메커니즘:**
1. **Representation Learning (DMF)**: User/Item의 latent factor를 독립적으로 학습
2. **Metric Learning (MLP)**: User-Item 상호작용을 직접 모델링
3. **Fusion (CFNet)**: 두 접근법을 결합하여 상호 보완

**Note:** 
- 위 결과는 전체 ML-1M 데이터셋 기준 (6040 users, 3706 items)
- 현재 노트북은 샘플 데이터(100 users)로 테스트하므로 성능 차이 발생 가능
- 전체 데이터셋으로 학습 시 논문과 유사한 성능 예상
- 샘플 데이터에서도 CFNet-pretrain이 가장 높은 성능을 보일 것으로 예상